## Inductance of cylindrical air-core inductor

Uses Nagaoka's Formula (Reference: H. Nagaoka, "The Inductance Coeficients of Solenoids," Journal of the College of Science, Imperial University, vol. 27, no. 6, 1909).

The formula uses scipy.special.ellipk and scipy.special.ellipe, integrals, which crash when passed Pint Quantities. So the `finunits` and `foutunits` parameters are passed to `UncertCalc` to define the "natural" units of the function - specifying that when the function recieves input arguments with units meter, meter, dimensionless, and Henry/meter, the output of the function will be Henry.

Because of the 2019 SI redefinition, vacuum permeability, $\mu_0$, has some uncertainty. To account for the uncertianty, mu0 is included as an input parameter rather than being hardcoded.

In [1]:
import numpy as np
from scipy.special import ellipk, ellipe
import suncal

def inductance_nagaoka(radius, length, N, mu0):
    ''' Calculate inductance using Nagaoka formula
    
        Parameters
        ----------
        radius: float
            Radius of coil
        length: float
            Length of coil
        N: float
            Total number of turns
        mu0: float
            Vacuum Permeability
    '''
    k = np.sqrt(4*radius**2 / (4*radius**2 + length**2))
    kprime = np.sqrt(1 - k**2)
    Kk = ellipk(k**2)
    Ek = ellipe(k**2)
    kL = 4/3/np.pi/kprime * ((kprime/k)**2 * (Kk - Ek) + Ek - k)
    return mu0 * np.pi * N**2 * radius**2 / length * kL

In [2]:
uc = suncal.UncertCalc(inductance_nagaoka, units='uH',
                       finunits=['m', 'm', '', 'H/m'], foutunits=['H'],
                       foutnames=['L'])
uc.set_input('radius', 2.7, units='mm', unc=.005, k=1)
uc.set_input('length', 9, units='mm', unc=.01, k=1)
uc.set_input('N', 100, units='dimensionless', unc=.1, k=1)
uc.set_input('mu0', 1.25663706212E-6, unc=0.00000000019E-6, units='H/m', k=2)
uc.calculate()

|Function   | Method       | Nominal   | Std. Uncertainty   | 95% Coverage   | k         | Deg. Freedom  |
|----------|------------|---------|------------------|--------------|---------|--------------|
|$L$   | GUM          | 25.22 µH  | 0.100 µH           | ± 0.20 µH     | 1.960     | inf           |
|$L$   | Monte Carlo  | 25.22 µH  | 0.100 µH           | (25.02 µH, 25.41 µH) | 1.959     | -             |

A sensitivity coefficient report, shown below, indicates that we probably didn't need to bother with accounting for the uncertainty in $\mu_0$.

In [3]:
uc.out.report_sens()

|Variable   | GUM Sensitivity   | GUM Proportion   | MC Sensitivity   | MC Proportion  |
|----------|-----------------|----------------|----------------|---------------|
|$N$   | 5.0e-07 H          | 25.56%           | 5.0e-07 H         | 25.62%         |
|$length$   | -2.2e-06 H/mm          | 4.83%            | 2.2e-06 H/mm         | 4.84%          |
|$\mu_{0}$   | 0.0 m          | 0.00%            | 20.1 m         | 0.00%          |
|$radius$   | 0.000017 H/mm          | 69.61%           | 0.000017 H/mm         | 69.60%         |